# Capstone Neighborhood Clustering Lab

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Acquire and Clean Data</a>

2.  <a href="#item2">Get lat and long data</a>

3.  <a href="#item3">Explore and Cluster Neighborhoods in Toronto</a>
    </font>
    </div>

## 1. Acquire and Clean Data

### Install necessary libraries and set options

In [1]:
# install necessary libraries and set options
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
#!pip install folium
#import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Install complete')

Install complete


In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium
print('Install complete')

Install complete


### Acquire the data by scraping the Wikipedia page and creating a new dataframe

In [3]:
# scrape data from Wikipedia page and store in dataframe
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802") 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0]  
df = pd.read_html(str(table)) 
neighborhood=pd.DataFrame(df[0])

In [4]:
# check the shape and first 10 rows of the dataframe prior to cleaning
print(neighborhood.shape)
neighborhood.head(10)

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Clean data

In [5]:
# rename Neighbourhood column to Neighborhood
neighborhood.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
# confirm how many rows contain "Not assigned" in the Borough column. These will be removed.
neighborhood[neighborhood['Borough'] == "Not assigned"].shape

(77, 3)

In [6]:
# create a new dataframe where rows with "Not assigned" in the Borough column have been removed.
neighborhood_filtered = neighborhood[neighborhood['Borough'] != "Not assigned"]
neighborhood_filtered.reset_index(drop = True, inplace = True)

In [7]:
# check the shape and first 10 rows of the new cleaned dataframe
print(neighborhood_filtered.shape)
neighborhood_filtered.head(10)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
# check to see if there are any rows with "Not assigned" in the Neighbourhood column
neighborhood_filtered[neighborhood_filtered['Neighborhood'] == "Not assigned"].shape

(0, 3)

In [9]:
# check to see if any postal codes are listed more than once
neighborhood_filtered.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,11,99
top,M9L,North York,Downsview
freq,1,24,4


In [10]:
neighborhood_filtered.shape

(103, 3)

## 2. Get lat and long data

### For each postal code insert the lat and long

In [11]:
latitude=[]
longitude=[]
for code in neighborhood_filtered['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    # print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        # print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

# create dataframes for the latitude and longitude lists
latitude_df = pd.DataFrame(latitude, columns = {'Latitude'})
longitude_df = pd.DataFrame(longitude, columns = {'Longitude'})

# merge the latitude and longitude dataframes with the neighborhood dataframe
lat_long_df = latitude_df.merge(longitude_df, how = 'left', left_index = True, right_index = True)
neighborhood_merged = neighborhood_filtered.merge(lat_long_df, how = 'left', left_index = True, right_index = True)

In [12]:
neighborhood_merged.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


## 3. Explore and Cluster Neighborhoods in Toronto

### Define Foursquare credentials

In [13]:
# define Foursqure credentials
CLIENT_ID = 'JK1IUMWSHEPA5Y5V4ZDWXOQ0I0HWQMMZ01OAKSLDUNKRSY5E' 
CLIENT_SECRET = 'NIAOW0W2NAAPK51TSLQPI5K1WPGGCXC0QVPIXVKCZMY13UWX' 
VERSION = '20180605'
LIMIT = 100 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JK1IUMWSHEPA5Y5V4ZDWXOQ0I0HWQMMZ01OAKSLDUNKRSY5E
CLIENT_SECRET:NIAOW0W2NAAPK51TSLQPI5K1WPGGCXC0QVPIXVKCZMY13UWX


### Create a function to look up venues from Foursquare for each lat and long pair

In [14]:
# create a function to get nearby venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues
toronto_venues = getNearbyVenues(names=neighborhood_merged['Neighborhood'],
                                   latitudes=neighborhood_merged['Latitude'],
                                   longitudes=neighborhood_merged['Longitude']
                                  )

In [16]:
# analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot.rename(columns={"Neighborhood": "Neighborhood Venue"}, inplace = True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.066667,0.000000,0.

### Define a function to sort the venues in descending order

In [18]:
# define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a new dataframe and display the top 10 venues for each neighborhood

In [19]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Skating Rink,Sushi Restaurant,Bakery,Supermarket,Badminton Court,Grocery Store,Newsagent,Discount Store
1,"Alderwood, Long Branch",Convenience Store,Gym,Dance Studio,Pub,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target
2,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Lawyer,Accessories Store,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Mexican Restaurant
3,Bayview Village,Trail,Park,Construction & Landscaping,Dog Run,Accessories Store,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Spa,Thai Restaurant,Pet Store,Pub,Comfort Food Restaurant,Restaurant,Fast Food Restaurant


### Cluster Neighborhoods by running k-means to cluster the neighborhood into 5 clusters

In [20]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 2, 2, 2, 2, 2, 2], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhood_merged_new = neighborhood_merged

# merge neighborhood data with venue data to add latitude/longitude for each neighborhood
neighborhood_merged_new = neighborhood_merged_new.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# drop rows that contain NaN values and convert Cluster Labels back to integers
neighborhood_merged_new.dropna(axis = 0, inplace = True)
neighborhood_merged_new = neighborhood_merged_new.astype({'Cluster Labels': 'int'})

neighborhood_merged_new.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,0,Food & Drink Shop,Park,Fast Food Restaurant,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant
1,M4A,North York,Victoria Village,43.73057,-79.31306,0,Park,German Restaurant,Grocery Store,Nightclub,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Accessories Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,2,Coffee Shop,Breakfast Spot,Yoga Studio,Pub,Spa,Event Space,Restaurant,Electronics Store,Bakery,Thai Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,2,Clothing Store,Women's Store,Restaurant,Men's Store,Cosmetics Shop,Bookstore,Toy / Game Store,Furniture / Home Store,Fried Chicken Joint,Coffee Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,2,Coffee Shop,Bank,Fried Chicken Joint,Mediterranean Restaurant,Café,Gastropub,Falafel Restaurant,Sandwich Place,Theater,Burrito Place


### Visualize the resulting clusters

In [22]:
#get lat and long for Toronto
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

In [23]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged_new['Latitude'], neighborhood_merged_new['Longitude'], neighborhood_merged_new['Neighborhood'], neighborhood_merged_new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each cluster

#### Cluster 0

In [24]:
neighborhood_merged_new.loc[neighborhood_merged_new['Cluster Labels'] == 0, neighborhood_merged_new.columns[[2] + list(range(5, neighborhood_merged_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,0,Food & Drink Shop,Park,Fast Food Restaurant,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant
1,Victoria Village,0,Park,German Restaurant,Grocery Store,Nightclub,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Accessories Store
5,"Islington Avenue, Humber Valley Village",0,Pharmacy,Grocery Store,Shopping Mall,Skating Rink,Park,Bank,Café,Accessories Store,Neighborhood Venue,Music Venue
16,Humewood-Cedarvale,0,Hockey Arena,Trail,Playground,Dog Run,Park,Field,Grocery Store,Neighborhood Venue,Music Venue,Museum
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,Park,College Rec Center,Shopping Mall,Grocery Store,Electronics Store,Fish & Chips Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Nightclub
18,"Guildwood, Morningside, West Hill",0,Park,Construction & Landscaping,Gym / Fitness Center,Tea Room,Accessories Store,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum
32,Scarborough Village,0,Spa,Indian Restaurant,Restaurant,Grocery Store,Park,Modern European Restaurant,Molecular Gastronomy Restaurant,Mobile Phone Shop,Monument / Landmark,Newsagent
35,"East Toronto, Broadview North (Old East York)",0,Park,Film Studio,Intersection,Accessories Store,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater
39,Bayview Village,0,Trail,Park,Construction & Landscaping,Dog Run,Accessories Store,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum
49,"North Park, Maple Leaf Park, Upwood Park",0,Bakery,Basketball Court,Park,Accessories Store,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target


#### Cluster 1

In [25]:
neighborhood_merged_new.loc[neighborhood_merged_new['Cluster Labels'] == 1, neighborhood_merged_new.columns[[2] + list(range(5, neighborhood_merged_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Hillcrest Village,1,Park,Residential Building (Apartment / Condo),Accessories Store,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Nightclub
45,"York Mills, Silver Hills",1,Park,Accessories Store,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Nightclub
68,"Forest Hill North & West, Forest Hill Road Park",1,Park,Accessories Store,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Nightclub
69,"High Park, The Junction South",1,Sandwich Place,Park,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Accessories Store,Nightclub
91,Rosedale,1,Park,Playground,Bike Trail,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark
98,"The Kingsway, Montgomery Road, Old Mill North",1,Lounge,Park,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Movie Theater,Accessories Store,Nightclub


#### Cluster 2

In [26]:
neighborhood_merged_new.loc[neighborhood_merged_new['Cluster Labels'] == 2, neighborhood_merged_new.columns[[2] + list(range(5, neighborhood_merged_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",2,Coffee Shop,Breakfast Spot,Yoga Studio,Pub,Spa,Event Space,Restaurant,Electronics Store,Bakery,Thai Restaurant
3,"Lawrence Manor, Lawrence Heights",2,Clothing Store,Women's Store,Restaurant,Men's Store,Cosmetics Shop,Bookstore,Toy / Game Store,Furniture / Home Store,Fried Chicken Joint,Coffee Shop
4,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Bank,Fried Chicken Joint,Mediterranean Restaurant,Café,Gastropub,Falafel Restaurant,Sandwich Place,Theater,Burrito Place
7,Don Mills,2,Coffee Shop,Burger Joint,Park,Bubble Tea Shop,Supermarket,Grocery Store,Gym,Gas Station,Smoke Shop,Intersection
8,"Parkview Hill, Woodbine Gardens",2,Pizza Place,Gym / Fitness Center,Pharmacy,Pet Store,Rock Climbing Spot,Fast Food Restaurant,Café,Flea Market,Gastropub,Bank
9,"Garden District, Ryerson",2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Italian Restaurant,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Ramen Restaurant,Theater
10,Glencairn,2,Pizza Place,Grocery Store,Japanese Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Gas Station,Latin American Restaurant,Bakery,Bank,Italian Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",2,Pizza Place,Tea Room,Sandwich Place,Chinese Restaurant,Pet Store,Pie Shop,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
12,"Rouge Hill, Port Union, Highland Creek",2,Bar,Accessories Store,Movie Theater,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Moving Target,Moroccan Restaurant,Metro Station
13,Don Mills,2,Coffee Shop,Burger Joint,Park,Bubble Tea Shop,Supermarket,Grocery Store,Gym,Gas Station,Smoke Shop,Intersection


#### Cluster 3

In [27]:
neighborhood_merged_new.loc[neighborhood_merged_new['Cluster Labels'] == 3, neighborhood_merged_new.columns[[2] + list(range(5, neighborhood_merged_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",3,Fast Food Restaurant,Accessories Store,Nightclub,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater


#### Cluster 4

In [28]:
neighborhood_merged_new.loc[neighborhood_merged_new['Cluster Labels'] == 4, neighborhood_merged_new.columns[[2] + list(range(5, neighborhood_merged_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,4,Home Service,Noodle House,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target
71,"Wexford, Maryvale",4,Home Service,Auto Garage,Moving Target,Newsagent,New American Restaurant,Neighborhood Venue,Music Venue,Museum,Movie Theater,Noodle House
